In [1]:
import numpy
import os
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import nn
from torch.nn import Sequential, Conv1d, MaxPool1d, Flatten, Linear,ReLU,Softmax,Tanh
from torch.utils.tensorboard import SummaryWriter

In [2]:
class MyData(Dataset):

    def __init__(self, root_dir, label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.data_path = os.listdir(self.path)

    def __getitem__(self, idx):
        txt_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, self.label_dir, txt_name)
        data = numpy.loadtxt(data_item_path)
        
        #list = []
        #data = list.append(data)
        
        label = self.label_dir
        return data, label

    def __len__(self):
        return len(self.data_path)

    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model1 = Sequential(
            Conv1d(1, 16, kernel_size=11),
            Conv1d(16, 16, kernel_size=3),
            Conv1d(16, 16, kernel_size=3),
            MaxPool1d(3),
            Conv1d(16, 64, kernel_size=3),
            Conv1d(64, 64,kernel_size=3),
            MaxPool1d(3),
            Conv1d(64, 64,kernel_size=3),
            Conv1d(64, 64,kernel_size=3),
            MaxPool1d(3),
            Flatten(),
            Linear(448, 64),
            Linear(64, 3)
        )

    def forward(self, x):
        x = self.model1(x)
        return x

In [3]:
train_data_dir = 'E:\\1D-Data\\train\\'
test_data_dir = 'E:\\1D-Data\\val\\'
bus_label_dir = 'bus'
car_label_dir = 'car'
man_label_dir = 'man'

In [4]:
train_bus_data = MyData(train_data_dir,bus_label_dir)
train_car_data = MyData(train_data_dir,car_label_dir)
train_man_data = MyData(train_data_dir,man_label_dir)
train_data = train_bus_data+train_car_data+train_man_data

In [5]:
test_bus_data = MyData(test_data_dir,bus_label_dir)
test_car_data = MyData(test_data_dir,car_label_dir)
test_man_data = MyData(test_data_dir,man_label_dir)
test_data = test_bus_data+test_car_data+test_man_data

In [6]:
Data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)  # 训练集
Data_loader_test = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)  # 测试集

In [7]:
# 获取数据集长度
dataset_train_size = len(Data_loader_train)
dataset_test_size = len(Data_loader_test)
print("训练数据集长度：{}".format(dataset_train_size))
print("测试数据集长度：{}".format(dataset_test_size))

训练数据集长度：7272
测试数据集长度：1560


In [8]:
OneD_nn = Net()

In [ ]:
# 损失函数
loss = nn.CrossEntropyLoss()

# 优化器
learn_rate = 1e-2
optimizer = torch.optim.SGD(OneD_nn.parameters(), lr=learn_rate)

# 设置训练网络参数
# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练的轮数
epoch = 30
# 添加tensorboard
#writer = SummaryWriter("log")

for i in range(epoch):
    print("----第 {} 轮训练开始----".format(i + 1))

    # 训练步骤开始
    OneD_nn.train()  # 有dropout和normlizer层需要调用
    for data in Data_loader_train:
        datas, targets = data
        
        list = []
        datas = datas.numpy().tolist()
        list.append(datas)
        datas=torch.Tensor(list)
        
        if targets == ('bus',):
            targets = [0]
        elif targets == ('car',):
            targets = [1]
        elif targets == ('man',):
            targets = [2] 
        
        targets = torch.LongTensor(targets)
        #print(datas)
        output_x = OneD_nn(datas)
        #print(targets)
        #print(output_x)
        loss_x = loss(output_x, targets)

        # 优化器优化模型
        optimizer.zero_grad()  # 梯度清0
        loss_x.backward()  # 求解grad梯度
        optimizer.step()  # 更新weight

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{} ".format(total_train_step, loss_x.item()))
            writer.add_scalar("train_loss", loss_x.item(), total_train_step)

    # 测试步骤开始
    OneD_nn.eval()  # 同理
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in Data_loader_test:
            datas, targets = data
            
            #datas = numpy.transpose(datas)
            list = []
            datas = datas.numpy().tolist()
            list.append(datas)
            datas=torch.Tensor(list)
            
            if targets == ('bus',):
                targets = [0]
            elif targets == ('car',):
                targets = [1]
            elif targets == ('man',):
                targets = [2] 
                
            targets = torch.LongTensor(targets)
            
            output_x = OneD_nn(datas)
            loss_x = loss(output_x, targets)
            total_test_loss = total_test_loss + loss_x.item()
            accuracy = (output_x.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
    print("整体测试集上的loss：{}".format(total_test_loss))
    print("整体测试集上的正确率：{}".format(total_accuracy / dataset_test_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy / dataset_test_size, total_test_step)
    total_test_step = total_test_step + 1
    if i == epoch-1:
        torch.save(OneD_nn, "OneD_nn_{}.pth".format(i))
    # torch.save(xjl_nn.state_dict(),"xjl_nn_{}.pth".format(i))
    print("模型已保存")

writer.close()


----第 1 轮训练开始----
训练次数：100，Loss：1.202393651008606 
训练次数：200，Loss：1.1847429275512695 
训练次数：300，Loss：1.0958410501480103 
训练次数：400，Loss：1.0684385299682617 
训练次数：500，Loss：1.0778298377990723 
训练次数：600，Loss：1.1022340059280396 
训练次数：700，Loss：1.078439474105835 
训练次数：800，Loss：1.1256210803985596 
训练次数：900，Loss：1.1742218732833862 
训练次数：1000，Loss：1.3868836164474487 
训练次数：1100，Loss：1.1868442296981812 
训练次数：1200，Loss：1.2239301204681396 
训练次数：1300，Loss：0.9959933161735535 
训练次数：1400，Loss：1.1901949644088745 
训练次数：1500，Loss：1.1657698154449463 
训练次数：1600，Loss：1.1236763000488281 
训练次数：1700，Loss：1.0651533603668213 
训练次数：1800，Loss：0.7356005311012268 
训练次数：1900，Loss：0.5926783680915833 
训练次数：2000，Loss：0.23225660622119904 
训练次数：2100，Loss：0.696358859539032 
训练次数：2200，Loss：0.08916682749986649 
训练次数：2300，Loss：0.7504448294639587 
训练次数：2400，Loss：0.1286058872938156 
训练次数：2500，Loss：0.31785768270492554 
训练次数：2600，Loss：0.5541101098060608 
训练次数：2700，Loss：0.041717056185007095 
训练次数：2800，Loss：0.8804062604904175 
训练次数：2900

训练次数：21600，Loss：2.50339189733495e-06 
训练次数：21700，Loss：3.4450891689630225e-05 
训练次数：21800，Loss：1.9073468138230965e-06 
整体测试集上的loss：3037.7943770034954
整体测试集上的正确率：0.6820513010025024
模型已保存
----第 4 轮训练开始----
训练次数：21900，Loss：0.0003636414185166359 
训练次数：22000，Loss：0.075171560049057 
训练次数：22100，Loss：9.572047565598041e-05 
训练次数：22200，Loss：0.001511027687229216 
训练次数：22300，Loss：0.00014184899919200689 
训练次数：22400，Loss：4.732496745418757e-05 
训练次数：22500，Loss：0.0003766304289456457 
训练次数：22600，Loss：2.622600959512056e-06 
训练次数：22700，Loss：1.1920928244535389e-07 
训练次数：22800，Loss：0.0 
训练次数：22900，Loss：0.00043930893298238516 
训练次数：23000，Loss：0.040020547807216644 
训练次数：23100，Loss：1.3708974620385561e-05 
训练次数：23200，Loss：0.0019094583112746477 
训练次数：23300，Loss：0.00646381126716733 
训练次数：23400，Loss：2.7418097943154862e-06 
训练次数：23500，Loss：5.960462772236497e-07 
训练次数：23600，Loss：0.00040570611599832773 
训练次数：23700，Loss：2.455681169521995e-05 
训练次数：23800，Loss：0.0014147283509373665 
训练次数：23900，Loss：0.0008092227508313954